In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/OSOP

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/OSOP


In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.11.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

--2022-07-06 17:32:05--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘plot_image_grid.py.15’

plot_image_grid.py. 100%[===================>]   1.57K  --.-KB/s    in 0.001s  

2022-07-06 17:32:05 (1.99 MB/s) - ‘plot_image_grid.py.15’ saved [1608/1608]



In [ ]:
import os
import numpy as np
from tqdm.notebook import tqdm
import imageio
import matplotlib.pyplot as plt
from skimage import img_as_ubyte
from PIL import Image

# io utils
from pytorch3d.io import load_obj, load_ply

# datastructures
from pytorch3d.structures import Meshes

# 3D transformations functions
from pytorch3d.transforms import Rotate, Translate

# rendering components
from pytorch3d.renderer import (
    FoVPerspectiveCameras, look_at_view_transform, look_at_rotation, 
    RasterizationSettings, MeshRenderer, MeshRasterizer, BlendParams,
    SoftSilhouetteShader, HardPhongShader, PointLights, TexturesVertex,
)

If you are running this notebook locally after cloning the PyTorch3D repository, the mesh will already be available. **If using Google Colab, fetch the mesh and save it at the path `data/`**:

In [ ]:
!mkdir -p data
!wget -P data https://dl.fbaipublicfiles.com/pytorch3d/data/teapot/teapot.obj

--2022-07-06 16:22:51--  https://dl.fbaipublicfiles.com/pytorch3d/data/teapot/teapot.obj
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152595 (149K) [text/plain]
Saving to: ‘data/teapot.obj.9’

teapot.obj.9        100%[===================>] 149.02K   239KB/s    in 0.6s    

2022-07-06 16:22:53 (239 KB/s) - ‘data/teapot.obj.9’ saved [152595/152595]



In [ ]:
%ls

data/                  plot_image_grid.py.13  plot_image_grid.py.6
models_cad/            plot_image_grid.py.14  plot_image_grid.py.7
plot_image_grid.py     plot_image_grid.py.15  plot_image_grid.py.8
plot_image_grid.py.1   plot_image_grid.py.2   plot_image_grid.py.9
plot_image_grid.py.10  plot_image_grid.py.3   __pycache__/
plot_image_grid.py.11  plot_image_grid.py.4   t-less_download.py
plot_image_grid.py.12  plot_image_grid.py.5


In [ ]:
# Set the cuda device 
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")



# Load the obj and ignore the textures and materials.
#verts, faces_idx, _ = load_obj("./data/teapot.obj")
#faces = faces_idx.verts_idx

verts, faces = load_ply("models_cad/obj_01.ply")

# Initialize each vertex to be white in color.
verts_rgb = torch.ones_like(verts)[None]  # (1, V, 3)
textures = TexturesVertex(verts_features=verts_rgb.to(device))

# Create a Meshes object for the teapot. Here we have only one mesh in the batch.
teapot_mesh = Meshes(
    verts=[verts.to(device)],   
    faces=[faces.to(device)], 
    textures=textures
)

In [ ]:
cameras = FoVPerspectiveCameras(device=device)
raster_settings = RasterizationSettings(
    image_size=256, 
    blur_radius=0.0, 
    faces_per_pixel=1, 
)
lights = PointLights(device=device, location=((2.0, 2.0, -2.0),))
phong_renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=cameras, 
        raster_settings=raster_settings
    ),
    shader=HardPhongShader(device=device, cameras=cameras, lights=lights)
)

In [ ]:
count = 0
batch_count = 0

In [ ]:
total_batch_size = 2900
total_elev = torch.linspace(0, 180, total_batch_size)
total_azim = torch.linspace(-180, 180, total_batch_size)

batch_size = 100
total_count = 0

while total_count <= total_batch_size:
  meshes = teapot_mesh.extend(batch_size)

  # Get a batch of viewing angles. 
  elev = total_elev[batch_count*batch_size:(batch_count+1)*batch_size]
  azim = total_elev[batch_count*batch_size:(batch_count+1)*batch_size]


  # All the cameras helper methods support mixed type inputs and broadcasting. So we can 
  # view the camera from the same distance and specify dist=100 as a float,
  # and then specify elevation and azimuth angles for each viewpoint as tensors. 
  R, T = look_at_view_transform(dist=100, elev=elev, azim=azim)
  cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

  # Move the light back in front of the cow which is facing the -z direction.
  lights.location = torch.tensor([[0.0, 0.0, -3.0]], device=device)

  images = phong_renderer(meshes, cameras=cameras, lights=lights)
  print("Saved batch_count: {} {} - {}".format(batch_count, batch_count*batch_size, (batch_count+1)*batch_size))
  batch_count += 1
  total_count += batch_size

  for image in images:
    img = image.cpu().numpy().squeeze()
    img = img[:, :, :3]
    img = np.array(img*255, dtype=np.uint8)
    im = Image.fromarray(img)
    im.save("Export/image_" + str(count) + ".png")
    count += 1


Saved batch_count: 0 0 - 100
Saved batch_count: 1 100 - 200
Saved batch_count: 2 200 - 300
Saved batch_count: 3 300 - 400
Saved batch_count: 4 400 - 500
Saved batch_count: 5 500 - 600
Saved batch_count: 6 600 - 700
Saved batch_count: 7 700 - 800
Saved batch_count: 8 800 - 900
Saved batch_count: 9 900 - 1000
Saved batch_count: 10 1000 - 1100
Saved batch_count: 11 1100 - 1200
Saved batch_count: 12 1200 - 1300
Saved batch_count: 13 1300 - 1400
Saved batch_count: 14 1400 - 1500
Saved batch_count: 15 1500 - 1600
Saved batch_count: 16 1600 - 1700
Saved batch_count: 17 1700 - 1800
Saved batch_count: 18 1800 - 1900
Saved batch_count: 19 1900 - 2000
Saved batch_count: 20 2000 - 2100
Saved batch_count: 21 2100 - 2200
Saved batch_count: 22 2200 - 2300
Saved batch_count: 23 2300 - 2400
Saved batch_count: 24 2400 - 2500
Saved batch_count: 25 2500 - 2600
Saved batch_count: 26 2600 - 2700
Saved batch_count: 27 2700 - 2800
Saved batch_count: 28 2800 - 2900


ValueError: ignored